In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K


from keras.utils import multi_gpu_model
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import tensorflow as tf

import gc

import time


%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True

hdf5_file = h5py.File(hdf5_path, "r")

if subtract_mean:
    mm = hdf5_file["train_mean"][...,0]
    mm = mm[np.newaxis, ...]

data_num = hdf5_file["train_flow"].shape[0]
    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

x_train = hdf5_file["train_flow"][...,0]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:,...]
hdf5_file.close()
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")


x_test = hdf5_file["IoT_flow"][...,0]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:,...]

hdf5_file.close()

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

#define number of packets used to train for each device
packets = 298
x_train = x_train[:,:packets,:]
x_test = x_test[:,:packets,:]

input_shape = (x_train.shape[1], x_train.shape[2])
    
    
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
AccScores = []
Times = []
LossScores = []
#AucScores = []
#F1Scores = []
#PrecisionScores = []
t2 = time.time()
for i in range (1,50):
    num_classes = 2
    high = 80
    
    filter_lenghts =  [int(i) for i in np.arange(2,high,2)]
    #print(filter_lenghts)
    convs = []
    maxlen = packets
    batch_size = 1024
    epochs = 30
    numFilters=128
    activations= "sigmoid"
    dropoutVal = 0.2485206320388983
    lr = 0.001362881122433337
    adam = keras.optimizers.Adam(lr=lr)


    optim = adam
    
    input_flow = Input(shape=input_shape)

    convs= {}
    mpoolings = {}
    flattens = {}
    convs_out = []
    for i in filter_lenghts:
        convs[str(i)+'_convolution']=Conv1D(filters=numFilters,kernel_size=i,padding="valid",activation=activations,strides=1)(input_flow)

        mpoolings[str(i)+'_maxpooling'] = MaxPooling1D(pool_size= maxlen - i + 1)(convs[str(i)+'_convolution'])
        flattens[str(i)+'_flattenout'] = Flatten()(mpoolings[str(i)+'_maxpooling'])
        convs_out.append(flattens[str(i)+'_flattenout'])
    out = Concatenate()(convs_out)
    dropout = Dropout(dropoutVal)(out)
    dense = Dense(64, activation='relu')(dropout)
    dense2 = Dense(32, activation='relu')(dense)
    dropout2 = Dropout(dropoutVal)(dense2)
    end = Dense(num_classes, activation='softmax')(dropout2)

    model = Model(inputs=input_flow, outputs=end) 
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
    
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    t0 = time.time()
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    t1 = time.time()
    total_run = t1-t0
    Times.append(total_run)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.5f%%" % (model.metrics_names[0], scores[0]))
    print('Fit and Evaluate Time:',total_run)
    AccScores.append(scores[1] * 100)
    LossScores.append(scores[0])
    del model
    gc.collect()
    K.clear_session()
    gc.collect()

    
print("%.5f%% (+/- %.5f%%)" % (np.mean(AccScores), np.std(AccScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(LossScores), np.std(LossScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(Times), np.std(Times)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(AucScores), np.std(AucScores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(F1Scores), np.std(F1Scores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(PrecisionScores), np.std(PrecisionScores)))
t3 = time.time()
total_time = t3-t2
print('Time to do everything:',total_time)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy: 89.89968%
loss: 0.25593%
Fit and Evaluate Time: 629.6137328147888
accuracy: 89.66964%
loss: 0.27932%
Fit and Evaluate Time: 620.3107504844666
accuracy: 87.73923%
loss: 0.27617%
Fit and Evaluate Time: 622.6156368255615
accuracy: 87.92895%
loss: 0.30644%
Fit and Evaluate Time: 620.0771634578705
accuracy: 87.65622%
loss: 0.32577%
Fit and Evaluate Time: 623.3133292198181
accuracy: 89.00087%
loss: 0.25954%
Fit and Evaluate Time: 620.1861221790314
accuracy: 89.16926%
loss: 0.25808%
Fit and Evaluate Time: 623.2509214878082
accuracy: 89.21669%
loss: 0.25581%
Fit and Evaluate Time: 622.4279153347015
accuracy: 89.35661%
loss: 0.25247%
Fit and Evaluate Time: 621.1825399398804
accuracy: 88.74475%
loss: 0.24845%
Fit and Evaluate Time: 623.616260766983
accuracy: 88.94633%
loss: 0.27470%
Fit and Evaluate Time: 619.6208770275116
accuracy: 88.31788%
loss: 0.29266%
Fit and Evaluate Time: 621.2087345

In [4]:
AccScores

[89.89968299865723,
 89.66964483261108,
 87.73922920227051,
 87.92895078659058,
 87.6562237739563,
 89.00087475776672,
 89.1692578792572,
 89.21668529510498,
 89.35660719871521,
 88.74475359916687,
 88.94633054733276,
 88.31787705421448,
 89.44672346115112,
 88.22776079177856,
 89.00087475776672,
 89.21194076538086,
 88.25147747993469,
 88.07598352432251,
 88.99376392364502,
 87.33132481575012,
 87.44753003120422,
 88.28704953193665,
 89.09810781478882,
 88.13527226448059,
 88.33447694778442,
 89.36135172843933,
 89.3068015575409,
 88.31550478935242,
 87.8957450389862,
 88.74712586402893,
 89.3708348274231,
 88.73763680458069,
 88.38902711868286,
 88.26807737350464,
 88.8846755027771,
 87.74160146713257,
 89.30917382240295,
 87.72262930870056,
 88.8917863368988,
 88.92261981964111,
 88.5099709033966,
 89.05779123306274,
 82.94163942337036,
 89.07202482223511,
 88.78269791603088,
 88.90601396560669,
 88.33684921264648,
 88.2064163684845,
 88.09732794761658]

In [5]:
LossScores

[0.25593475341601807,
 0.27932078153680195,
 0.27616908617080377,
 0.30643616261680584,
 0.32576677804585696,
 0.25954075623456047,
 0.2580797931556951,
 0.2558139339126499,
 0.25247172338671026,
 0.2484520538020406,
 0.27470413344004985,
 0.2926557879335476,
 0.25481620887200984,
 0.27021681278103665,
 0.2570116125651671,
 0.2533733333886881,
 0.2786863041232365,
 0.3078171561777155,
 0.25344664179127735,
 0.30415420046414854,
 0.30682559471367055,
 0.27912531950338254,
 0.25731988630920827,
 0.2987485867711301,
 0.28577206744850747,
 0.24558426476231388,
 0.2528004616514496,
 0.27090228943310013,
 0.2810084870725232,
 0.27410961023838215,
 0.26262793515588656,
 0.2706584228715064,
 0.28455346995084235,
 0.28403646209373,
 0.2597133680209113,
 0.28645853526650156,
 0.25667114041414013,
 0.3256088976401551,
 0.2841322182033621,
 0.2633866752672843,
 0.26999710821184064,
 0.25949050475079877,
 2.6234215325208003,
 0.258677962333,
 0.276101620693144,
 0.269007702988792,
 0.28707863962151

In [6]:
Times

[629.6137328147888,
 620.3107504844666,
 622.6156368255615,
 620.0771634578705,
 623.3133292198181,
 620.1861221790314,
 623.2509214878082,
 622.4279153347015,
 621.1825399398804,
 623.616260766983,
 619.6208770275116,
 621.2087345123291,
 622.788557767868,
 620.9104096889496,
 619.7083284854889,
 622.0011751651764,
 622.5200436115265,
 620.8299691677094,
 620.4627447128296,
 625.0748779773712,
 620.6119320392609,
 621.6898891925812,
 620.4228110313416,
 625.198714017868,
 621.8754072189331,
 624.0733504295349,
 625.6214809417725,
 622.6613495349884,
 622.5787613391876,
 622.4239296913147,
 622.9502580165863,
 621.0026273727417,
 619.6226735115051,
 622.4499115943909,
 625.0812470912933,
 623.220333814621,
 619.8093776702881,
 624.0431084632874,
 622.6283164024353,
 623.0565440654755,
 622.4008305072784,
 621.4474949836731,
 620.0726134777069,
 624.2259356975555,
 621.9556119441986,
 620.7188711166382,
 621.8701477050781,
 619.9172422885895,
 624.5187747478485]